In [2]:
import pandas as pd
import numpy as np

from keras import models
from keras import layers
from keras import optimizers

from keras_preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121 as BASE

from sklearn.metrics import roc_auc_score, average_precision_score

Using TensorFlow backend.


In [3]:
# -----------------------------LOAD IN OUR DATA---------------------------------- #
VALID = 'CheXpert-v1.0-small/valid.csv'
WEIGHTS = 'DenseNet121_16_6_weights_lr_reduce_from32.hdf5'

# generate validation set
valid = pd.read_csv(VALID, dtype=str)

#mapping to different labels
label = {'0.0': '0', '1.0' : '1', '-1.0' : '1'}

#same for validation set even though there should be no issue here with missing data
valid['label'] = valid['Pleural Effusion'].map(label)
num_valid = len(valid)

print(valid['label'].value_counts())


0    167
1     67
Name: label, dtype: int64


In [10]:
valid.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,label
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
#---------------------------------CREATE TEST DATA GENERATOR---------------------------------#
def test_flow(valid, target_size, batch_size = 1, color_mode="grayscale",  x = 'Path', y= 'label'):

    #declare the datagen options
    valid_datagen = ImageDataGenerator(rescale=1./255)

    valid_generator = valid_datagen.flow_from_dataframe(dataframe=valid,
                                                        directory=None,
                                                        x_col=x,
                                                        y_col=y,
                                                        class_mode="binary",
                                                        color_mode=color_mode,
                                                        target_size=target_size,
                                                        batch_size=batch_size)

    return valid_generator


valid_generator = test_flow(valid, (224,224), color_mode="rgb")

Found 234 validated image filenames belonging to 2 classes.


In [5]:
# # -----------------------------LOAD MODEL---------------------------------- #
conv_base = BASE(include_top=True,
                input_shape=valid_generator.image_shape,
                pooling=max)

conv_base.layers.pop()
# print(conv_base.summary())

# conv_base.trainable = True
# for layer in conv_base.layers:
#     layer.trainable = True

model = models.Sequential()
model.add(conv_base)
model.add(layers.Dense(1, activation='sigmoid'))

if WEIGHTS:
    model.load_weights(WEIGHTS)

model.compile(loss='binary_crossentropy', optimizer= optimizers.Adam(), metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1000)              7037504   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1001      
Total params: 7,038,505
Trainable params: 6,954,857
Non-trainable params: 83,648
_________________________________________________________________
None


In [40]:
scoreSeg = model.evaluate_generator(valid_generator, steps=len(valid), workers=0)
print('--------------------------')
print('')
print("Accuracy (Evaluation Generator)= ",scoreSeg[1])
print('')

--------------------------

Accuracy (Evaluation Generator)=  0.8675213675213675



In [7]:
y_pred_keras = model.predict_generator(valid_generator, steps=len(valid_generator.labels), verbose=1, workers=0)

234/234 [==============================] - 58s 247ms/step


In [9]:
valid['y_prob'] = y_pred_keras

valid['y_pred'] = round(valid['y_prob'])

valid['y_gen_label'] = valid_generator.labels

pd.crosstab(valid['label'], valid['y_pred'])


y_pred,0.0,1.0
label,,
0,131,36
1,51,16


At this point I am lost on how to do this. Likely need to develop my own generator? Or perhaps my training data is all wrong? I have lost all faith...

In [16]:
model.class_indices

AttributeError: 'Sequential' object has no attribute 'class_indices'